In [1]:
%%capture
%pip install llama-index-llms-huggingface

In [2]:
%%capture
!pip install -q pypdf llama-index einops accelerate python-dotenv bitsandbytes -U

In [3]:
%%capture
!pip install "transformers[torch]" "huggingface_hub[inference]"

In [4]:
%%capture
!pip install llama-index-llms-llama-cpp

In [5]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-05-16 08:38:17--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  13.7MB/s    in 1.2s    

2024-05-16 08:38:19 (13.7 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [6]:
import torch
import nest_asyncio
from llama_index.core import SimpleDirectoryReader
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.legacy.embeddings import HuggingFaceEmbedding
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.selectors import LLMMultiSelector
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
from transformers import BitsAndBytesConfig

nest_asyncio.apply()

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
##Load the data
documents = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()

##split the document
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

##Load the llm and the embedder
Settings.llm = LlamaCPP(
   model_url='https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf',
   model_path=None,
   temperature=0.1,
   max_new_tokens=256,
   context_window=3900,
   generate_kwargs={},
   model_kwargs={"n_gpu_layers":-1},
   verbose=True
)

Settings.embed_model = HuggingFaceEmbedding('thenlper/gte-large')

total size (MB): 4081.0


3892it [00:23, 163.50it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-16
last_modified_date: 2024-05-16

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [9]:
vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [10]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?",
)


llama_print_timings:        load time =  207837.60 ms
llama_print_timings:      sample time =      75.89 ms /   120 runs   (    0.63 ms per token,  1581.15 tokens per second)
llama_print_timings: prompt eval time =  347835.12 ms /   849 tokens (  409.70 ms per token,     2.44 tokens per second)
llama_print_timings:        eval time =   83806.02 ms /   119 runs   (  704.25 ms per token,     1.42 tokens per second)
llama_print_timings:       total time =  432169.53 ms /   968 tokens


In [11]:
print(str(response))


MetaGPT achieves a new state-of-the-art (SoTA) with 85.9% and 87.7% in Pass@1 in code generation benchmarks, surpassing other popular frameworks for creating complex software projects such as AutoGPT, LangChain, AgentVerse, and ChatDev. Additionally, MetaGPT stands out in handling higher levels of software complexity and offers extensive functionality, with a 100% task completion rate in experimental evaluations, demonstrating its robustness and efficiency (time and token costs).


In [12]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-16', 'last_modified_date': '2024-05-16'}
